In [38]:
%run utils.ipynb
import matplotlib.pyplot as plt
import matplotlib.colors
import numpy as np
import os
import pandas as pd

In [39]:
figsize = (10,8)
dpi = 100

# All Graphs

In [43]:
def show_all_graphs(df, dataset_name, use_difference_psi, bp_threshold):
    show_metrics(df)
    show_ranking_dist(df)
    show_ranking_dist_by_direction(df)
    show_difference_psi_dist(df)
    show_difference_psi_dist_by_direction(df)    
    show_performance_different_thresholds(df, use_difference_psi)
    show_relation_frequency(dataset_name)
    show_better_predicted_per_relation(df, use_difference_psi, bp_threshold)
    show_better_predicted_per_relation_with_frequency(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_relation_with_frequency_swaped(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_relation_frequency_bin(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_relation_with_relation_classes(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_relation_with_relation_classes_direction(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_triples_per_relation_class_table(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_triples_per_relation_class_table_cleaned(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_by_similar_triple_exists(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_by_similar_triple_exists_steps(df, dataset_name, use_difference_psi, bp_threshold)
    show_entity_frequency(dataset_name)
    show_better_predicted_per_head_frequency_bin(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_tail_frequency_bin(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_combined_entity_frequency_bin(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_head_with_sim_score(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_head_with_sim_score_swaped(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_tail_with_sim_score(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_tail_with_sim_score_swaped(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_head_question_answer_entity_with_sim_score(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_tail_question_answer_entity_with_sim_score(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_head_cluster(df, dataset_name, use_difference_psi, bp_threshold, 10)
    show_better_predicted_per_head_cluster(df, dataset_name, use_difference_psi, bp_threshold, 25)
    show_better_predicted_per_head_cluster(df, dataset_name, use_difference_psi, bp_threshold, 100)
    #show_better_predicted_per_head_cluster_swaped(df, dataset_name, use_difference_psi, bp_threshold, 10)
    show_better_predicted_per_tail_cluster(df, dataset_name, use_difference_psi, bp_threshold, 10)
    show_better_predicted_per_tail_cluster(df, dataset_name, use_difference_psi, bp_threshold, 25)
    show_better_predicted_per_tail_cluster(df, dataset_name, use_difference_psi, bp_threshold, 100)
    #show_better_predicted_per_tail_cluster_swaped(df, dataset_name, use_difference_psi, bp_threshold, 10)
    show_better_predicted_per_question_cluster(df, dataset_name, use_difference_psi, bp_threshold, 10)
    show_better_predicted_per_question_cluster(df, dataset_name, use_difference_psi, bp_threshold, 25)
    show_better_predicted_per_question_cluster(df, dataset_name, use_difference_psi, bp_threshold, 100)
    show_better_predicted_per_answer_cluster(df, dataset_name, use_difference_psi, bp_threshold, 10)
    show_better_predicted_per_answer_cluster(df, dataset_name, use_difference_psi, bp_threshold, 25)
    show_better_predicted_per_answer_cluster(df, dataset_name, use_difference_psi, bp_threshold, 100)

def show_all_graphs_wo_sim_score(df, dataset_name, use_difference_psi, bp_threshold):
    show_metrics(df)
    show_ranking_dist(df)
    show_ranking_dist_by_direction(df)
    show_difference_psi_dist(df)
    show_difference_psi_dist_by_direction(df)    
    show_performance_different_thresholds(df, use_difference_psi)
    show_relation_frequency(dataset_name)
    show_better_predicted_per_relation(df, use_difference_psi, bp_threshold)
    show_better_predicted_per_relation_with_frequency(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_relation_with_frequency_swaped(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_relation_frequency_bin(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_relation_with_relation_classes(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_relation_with_relation_classes_direction(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_triples_per_relation_class_table(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_triples_per_relation_class_table_cleaned(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_by_similar_triple_exists(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_by_similar_triple_exists_steps(df, dataset_name, use_difference_psi, bp_threshold)
    show_entity_frequency(dataset_name)
    show_better_predicted_per_head_frequency_bin(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_tail_frequency_bin(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_combined_entity_frequency_bin(df, dataset_name, use_difference_psi, bp_threshold)
    #show_better_predicted_per_head_with_sim_score(df, dataset_name, use_difference_psi, bp_threshold)
    #show_better_predicted_per_head_with_sim_score_swaped(df, dataset_name, use_difference_psi, bp_threshold)
    #show_better_predicted_per_tail_with_sim_score(df, dataset_name, use_difference_psi, bp_threshold)
    #show_better_predicted_per_tail_with_sim_score_swaped(df, dataset_name, use_difference_psi, bp_threshold)
    #show_better_predicted_per_head_question_answer_entity_with_sim_score(df, dataset_name, use_difference_psi, bp_threshold)
    #show_better_predicted_per_tail_question_answer_entity_with_sim_score(df, dataset_name, use_difference_psi, bp_threshold)
    show_better_predicted_per_head_cluster(df, dataset_name, use_difference_psi, bp_threshold, 10)
    show_better_predicted_per_head_cluster(df, dataset_name, use_difference_psi, bp_threshold, 25)
    show_better_predicted_per_head_cluster(df, dataset_name, use_difference_psi, bp_threshold, 100)
    show_better_predicted_per_head_cluster_swaped(df, dataset_name, use_difference_psi, bp_threshold, 10)
    show_better_predicted_per_tail_cluster(df, dataset_name, use_difference_psi, bp_threshold, 10)
    show_better_predicted_per_tail_cluster(df, dataset_name, use_difference_psi, bp_threshold, 25)
    show_better_predicted_per_tail_cluster(df, dataset_name, use_difference_psi, bp_threshold, 100)
    show_better_predicted_per_tail_cluster_swaped(df, dataset_name, use_difference_psi, bp_threshold, 10)

# Helper Functions

In [ ]:
def plt_show(pad=None):
    if pad:
        pdf.savefig(pad_inches=pad, bbox_inches="tight")
    else:
        pdf.savefig()
    plt.show()
    
def get_better_predicted_by(df, use_difference_psi, threshold):
    if use_difference_psi:
        df['psi_abs'] = df['difference_psi'] - 1
        t = threshold / 100
        df[f'better_predicted_by_anyburl_t{threshold}'] = df.apply(lambda x: -1 if x['psi_abs'] < (-1 * t) else 1 if x['psi_abs'] > t else 0, axis=1)        
        
        return df.drop(columns=['psi_abs'])
    else:
        df['rank_difference'] = df['rank_filtered_anyburl'] - df['rank_filtered_kge']
        df[f'better_predicted_by_anyburl_t{threshold}'] = df.apply(lambda x: 1 if x['rank_difference'] < (-1 * threshold) else -1 if x['rank_difference'] > threshold else 0, axis=1)
    
        return df.drop(columns=['rank_difference'])

def group_df_by_r_id(df):
    return df[['r_id', f'better_predicted_by_anyburl_t{bp_threshold}']].groupby('r_id').mean()

def group_df_by_r_id_and_predicted_head(df):
    return df[['r_id', 'predicted_head', f'better_predicted_by_anyburl_t{bp_threshold}']].groupby(['r_id', 'predicted_head']).mean()

def group_df_by_h_id(df):
    return df[['h_id', f'better_predicted_by_anyburl_t{bp_threshold}']].groupby('h_id').mean()

def group_df_by_t_id(df):
    return df[['t_id', f'better_predicted_by_anyburl_t{bp_threshold}']].groupby('t_id').mean()

def get_title_key():
    return f'{dataset_name}, {symbolic_name}, {subsymbolic_name}'

# Show Metrics

In [ ]:
def show_metrics(df):    
    models = [
        (symbolic_name, 'rank_filtered_anyburl'),
        (f'{subsymbolic_name} Combined', 'rank_filtered_kge'),
        (f'{subsymbolic_name} 1', 'rank_filtered_kge_0'),
        (f'{subsymbolic_name} 2', 'rank_filtered_kge_1'),
        (f'{subsymbolic_name} 3', 'rank_filtered_kge_2'),
        (f'{subsymbolic_name} 4', 'rank_filtered_kge_3'),
        (f'{subsymbolic_name} 5', 'rank_filtered_kge_4')
    ]             
    metrics = [
        ('MRR', calculate_mean_reciprocal_rank),
        ('Hits@1', calculate_hits_at_1),
        ('Hits@10', calculate_hits_at_10),
        ('Hits@100', calculate_hits_at_100)
    ] 
    
    metrics_df = pd.DataFrame(columns=[metric[0] for metric in metrics])
                   
    for model in models:       
        metrics_df.loc[model[0]] = [f'{metric[1](df[model[1]]):.3f}' for metric in metrics]
    
    fig, ax = plt.subplots(dpi=dpi)
    table = ax.table(colLabels=metrics_df.columns, rowLabels=metrics_df.index, loc='center', cellText=metrics_df.values)
        
    fig.suptitle(f'({get_title_key()}) Metrics')
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')
    
    table.auto_set_font_size(True)

    plt_show(pad=1)

# Boxplots of ranking distribution

In [ ]:
def show_ranking_dist(df):
    """
    expects a df with 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    plt.figure(figsize=figsize, dpi=dpi)
    plt.boxplot(df[['rank_filtered_anyburl', 'rank_filtered_kge']], labels=[symbolic_name, subsymbolic_name])
    
    plt.ylabel('Rank (filtered)')
    plt.title(f'({get_title_key()}) Comparison of Rankings per Model')
    plt_show()

# Boxplots of ranking distribution by direction

In [ ]:
def show_ranking_dist_by_direction(df):
    """
    expects a df with 'rank_filtered_anyburl', 'rank_filtered_kge' and 'predicted_head'
    """
    heads = df[df['predicted_head']==True][['h_id', 'r_id', 't_id', 'rank_filtered_anyburl', 'rank_filtered_kge']]
    tails = df[df['predicted_head']==False][['h_id', 'r_id', 't_id','rank_filtered_anyburl', 'rank_filtered_kge']]
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize, dpi=dpi)    
    ax1.boxplot(heads[['rank_filtered_anyburl', 'rank_filtered_kge']], labels=[symbolic_name, subsymbolic_name])
    ax2.boxplot(tails[['rank_filtered_anyburl', 'rank_filtered_kge']], labels=[symbolic_name, subsymbolic_name])
    
    f.suptitle(f'({get_title_key()}) Comparison of Rankings per Model and Prediction Direction')
    ax1.set_title('Predicted Heads')
    ax2.set_title('Predicted Tails')
    ax1.set_ylabel('Rank (filtered)')
    ax2.set_ylabel('Rank (filtered)')

    plt_show()

# Boxplot of difference psi distribution

In [ ]:
def show_difference_psi_dist(df):
    """
    expects a df with 'difference_psi'
    """    
    plt.figure(figsize=figsize, dpi=dpi)
    plt.boxplot(df['difference_psi'])
    
    plt.ylabel('Difference Psi')
    plt.title(f'({get_title_key()}) Comparison of Rankings per Model')
    plt.ylim(0,2)
    plt_show()

# Boxplots of difference psi by direction

In [ ]:
def show_difference_psi_dist_by_direction(df):
    """
    expects a df with 'difference_psi' and 'predicted_head'
    """
    heads = df[df['predicted_head']==True][['h_id', 'r_id', 't_id', 'difference_psi']]
    tails = df[df['predicted_head']==False][['h_id', 'r_id', 't_id','difference_psi']]
    
    plt.figure(figsize=figsize, dpi=dpi)
    plt.boxplot([heads['difference_psi'], tails['difference_psi']], labels=['Predicted Heads', 'Predicted Tails'])
    
    plt.ylabel('Difference Psi')
    plt.title(f'({get_title_key()}) Comparison of Rankings per Model')
    plt.ylim(0,2)
    plt_show()

# Performance with different comparison Thresholds

In [ ]:
def show_performance_different_thresholds(df, use_difference_psi, rank_thresholds = [0,5,10,25,50,100]):
    """
    expects a df with 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """    
    for t in rank_thresholds:
        df = get_better_predicted_by(df, use_difference_psi, t)
    
    X_axis = np.arange(6)
    Y_anyburl = [(df[f'better_predicted_by_anyburl_t{t}'] == 1).sum() for t in rank_thresholds]
    Y_kge = [(df[f'better_predicted_by_anyburl_t{t}'] == -1).sum() for t in rank_thresholds]
    Y_equal = [(df[f'better_predicted_by_anyburl_t{t}'] == 0).sum() for t in rank_thresholds]

    plt.figure(figsize=figsize, dpi=dpi)
    plt.bar(X_axis - 0.2, Y_anyburl, 0.2, label = symbolic_name)
    plt.bar(X_axis, Y_kge, 0.2, label = subsymbolic_name)
    plt.bar(X_axis + 0.2, Y_equal, 0.2, label = 'Equal')
    
    plt.xticks(X_axis, rank_thresholds)
    plt.xlabel("Threshold")
    plt.ylabel("Amount of better Predictions")
    plt.title(f'({get_title_key()}) Comparison of Rankings for different Thresholds')
    plt.legend()
    
    plt_show()

# Relation Frequency

In [ ]:
def show_relation_frequency(dataset_name):
    r_freq = get_relation_frequency_in_training_data(dataset_name)
    
    plt.figure(figsize=figsize, dpi=dpi)
    plt.bar(r_freq['r_id'], r_freq['freq'])
    
    plt.xticks(r_freq['r_id'])
    plt.title(f'({get_title_key()}) Relation Frequency in Trainings Data')
    
    plt.xlabel('Relation Id')
    plt.ylabel('Amount of Occurrences in Trainings Data')
    
    plt_show()

# Entity Frequency

In [ ]:
def show_entity_frequency(dataset_name):    
    ht_freq = get_entity_frequency_in_training_data(dataset_name)
    plt.figure(figsize=figsize, dpi=dpi)
    plt.bar(ht_freq['h_id'], ht_freq['freq'])
    
    plt.xticks(ht_freq['h_id'])
    plt.title(f'({get_title_key()}) Entity Frequency in Trainings Data')
    
    plt.xlabel('Entity Id')
    plt.ylabel('Amount of Occurrences in Trainings Data')
    
    plt_show()

# Better Predicted by AnyBURL per Relation

In [ ]:
def show_better_predicted_per_relation(df, use_difference_psi, bp_threshold):
    """
    expects a df with 'r_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    
    df_by_r = group_df_by_r_id(df).sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    df_by_r.reset_index(inplace=True)
    
    plt.figure(figsize=figsize, dpi=dpi)
    plt.barh(range(len(df_by_r['r_id'])),  df_by_r[f'better_predicted_by_anyburl_t{bp_threshold}'])
    
    plt.yticks(range(len(df_by_r['r_id'])), df_by_r['r_id'])
    plt.xlabel('Better predicted by AnyBURL')
    plt.ylabel('Relation Id')
    plt.title(f'({get_title_key()}) Better Predicted by AnyBURL per Relation')
    plt.xlim(-1,1)
    
    plt_show()

# Better Predicted by AnyBURL per Relation with Frequency

In [ ]:
def show_better_predicted_per_relation_with_frequency(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 'r_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    r_freq = get_relation_frequency_in_training_data(dataset_name)
    
    df_by_r = group_df_by_r_id(df)
    df_by_r_w_freq = r_freq.join(df_by_r, on='r_id').sort_values('freq')
    df_by_r_w_freq.reset_index(inplace=True)
    df_by_r_w_freq = df_by_r_w_freq.dropna()
    df_by_r_w_freq = df_by_r_w_freq.sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    
    cmap = plt.cm.viridis
    norm = matplotlib.colors.Normalize(vmin=df_by_r_w_freq['norm_freq'].min(), vmax=df_by_r_w_freq['norm_freq'].nlargest(30).values[1])

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.bar(range(len(df_by_r_w_freq['r_id'])), df_by_r_w_freq[f'better_predicted_by_anyburl_t{bp_threshold}'], color=cmap(norm(df_by_r_w_freq['norm_freq'].values)))
    
    ax.set_xticks(range(len(df_by_r_w_freq['r_id'])), df_by_r_w_freq['r_id'])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Normalized Relation Frequency')

    plt.title(f'({get_title_key()}) Better predicted by AnyBURL per Relation with Relation Frequency in Trainings Data')
    plt.xlabel('Relation Id')
    plt.ylabel('Better predicted by AnyBURL')
    plt.ylim(-1,1)

    plt_show()

In [ ]:
def show_better_predicted_per_relation_with_frequency_swaped(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 'r_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    r_freq = get_relation_frequency_in_training_data(dataset_name)
    
    df_by_r = group_df_by_r_id(df)
    df_by_r_w_freq = r_freq.join(df_by_r, on='r_id').sort_values('freq')
    df_by_r_w_freq.reset_index(inplace=True)
    df_by_r_w_freq = df_by_r_w_freq.dropna()
    df_by_r_w_freq = df_by_r_w_freq.sort_values(['freq', f'better_predicted_by_anyburl_t{bp_threshold}'])
    
    cmap = plt.cm.RdYlGn
    norm = matplotlib.colors.Normalize(vmin=-1, vmax=1)

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.bar(range(len(df_by_r_w_freq['r_id'])), df_by_r_w_freq['norm_freq'], color=cmap(norm(df_by_r_w_freq[f'better_predicted_by_anyburl_t{bp_threshold}'].values)))
    
    ax.set_xticks(range(len(df_by_r_w_freq['r_id'])), df_by_r_w_freq['r_id'])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Better predicted by AnyBURL')

    plt.title(f'({get_title_key()}) Relation Frequency in Trainings Data with Better predicted by AnyBURL per Relation')
    plt.xlabel('Relation Id')
    plt.ylabel('Normalized Relation Frequency')
    plt.ylim(0,1)

    plt_show()

# Better Predicted by AnyBURL per Relation Frequency Bin

In [ ]:
def show_better_predicted_per_relation_frequency_bin(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 'r_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    r_freq = get_relation_frequency_in_training_data(dataset_name)
    
    df_w_freq = df.join(r_freq.set_index('r_id'), on='r_id', how='left')
    df_w_freq['bin'] = pd.cut(df_w_freq['norm_freq'], 10, retbins=True)[0]
    df_w_freq_by_bin = df_w_freq[['bin', f'better_predicted_by_anyburl_t{bp_threshold}']].groupby('bin').mean()
    df_w_freq_by_bin.reset_index(inplace=True)
    
    plt.figure(figsize=figsize, dpi=dpi)
    plt.barh(range(len(df_w_freq_by_bin['bin'])),  df_w_freq_by_bin[f'better_predicted_by_anyburl_t{bp_threshold}'])
    
    plt.yticks(range(len(df_w_freq_by_bin['bin'])), df_w_freq_by_bin['bin'])
    plt.xlabel('Better predicted by AnyBURL')
    plt.ylabel('Normalized Relation Frequency Bin')
    plt.title(f'({get_title_key()}) Better Predicted by AnyBURL per Normalized Relation Frequency Bin')
    plt.xlim(-1,1)
    
    plt_show()

# Correlation Entity Frequency Difference Psi

In [ ]:
def show_correlation_between_relation_frequency_and_difference_psi(df, dataset_name):
    """
    expects a df with 'r_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    r_freq = get_relation_frequency_in_training_data(dataset_name)  
    df_w_freq = df.join(r_freq.set_index('r_id'), on='r_id', how='left')
            
    print("Pearson correlation between Relation Frequency and Difference Psi = {df_w_freq['norm_freq'].corr(df_w_freq['difference_psi'])}")
    pdf.attach_note(f"Pearson correlation between Relation Frequency and Difference Psi = {df_w_freq['norm_freq'].corr(df_w_freq['difference_psi'])}")

# Better Predicted by AnyBURL per Relation with Relation Class

In [ ]:
def show_better_predicted_per_relation_with_relation_classes(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 'r_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    df_by_r = group_df_by_r_id(df).sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    relation_classes = get_relation_classes(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt')
    )
    df_by_r_w_classes = df_by_r.join(relation_classes)

    colors = {'1-1': 'red', '1-N': 'green', 'M-1': 'blue', 'M-N': 'orange'}
    c = df_by_r_w_classes['relationClass'].apply(lambda x: colors[x])

    plt.figure(figsize=figsize, dpi=dpi)
    plt.barh(range(len(df_by_r_w_classes['r_id'])),  df_by_r_w_classes[f'better_predicted_by_anyburl_t{bp_threshold}'], color=c)

    for i, j in colors.items(): 
        plt.barh(range(len(df_by_r_w_classes['r_id'])), df_by_r_w_classes[f'better_predicted_by_anyburl_t{bp_threshold}'],height=0,color=j,label=i)   
    
    plt.yticks(range(len(df_by_r_w_classes['r_id'])), df_by_r_w_classes['r_id'])    
    plt.title(f'({get_title_key()}) Better predicted by AnyBURL per Relation with Relation Class')
    plt.xlabel('Better predicted by AnyBURL')
    plt.ylabel('Relation Id')
    plt.xlim(-1,1)   
    
    plt.legend()
    plt_show()
    
def show_better_predicted_per_relation_with_relation_classes_direction(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 'r_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize, dpi=dpi)    
    
    f.suptitle(f'({get_title_key()}) Better predicted by AnyBURL per Relation with Relation Class')
    ax1.set_title('Predicted Heads')
    ax2.set_title('Predicted Tails')
    ax1.set_ylabel('Relation Id')
    ax2.set_ylabel('Relation Id')
    ax1.set_xlim(-1,1)
    ax2.set_xlim(-1,1)
    ax1.set_xlabel('Better predicted by AnyBURL')
    ax2.set_xlabel('Better predicted by AnyBURL')
        
    df_by_r = group_df_by_r_id(df[df['predicted_head'] == True]).sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    relation_classes = get_relation_classes(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt')
    )
    df_by_r_w_classes = df_by_r.join(relation_classes)
    colors = {'1-1': 'red', '1-N': 'green', 'M-1': 'blue', 'M-N': 'orange'}
    c = df_by_r_w_classes['relationClass'].apply(lambda x: colors[x])

    #ax1.figure(figsize=figsize, dpi=dpi)
    ax1.barh(range(len(df_by_r_w_classes['r_id'])),  df_by_r_w_classes[f'better_predicted_by_anyburl_t{bp_threshold}'], color=c)

    ax1.set_yticks(range(len(df_by_r_w_classes['r_id'])), df_by_r_w_classes['r_id'])
    for i, j in colors.items(): 
        ax1.barh(range(len(df_by_r_w_classes['r_id'])), df_by_r_w_classes[f'better_predicted_by_anyburl_t{bp_threshold}'],height=0,color=j,label=i)   
        
    ax1.legend()
    
    df_by_r = group_df_by_r_id(df[df['predicted_head'] == False]).sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    relation_classes = get_relation_classes(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt')
    )
    df_by_r_w_classes = df_by_r.join(relation_classes)
    colors = {'1-1': 'red', '1-N': 'green', 'M-1': 'blue', 'M-N': 'orange'}
    c = df_by_r_w_classes['relationClass'].apply(lambda x: colors[x])

    #ax2.figure(figsize=figsize, dpi=dpi)
    ax2.barh(range(len(df_by_r_w_classes['r_id'])),  df_by_r_w_classes[f'better_predicted_by_anyburl_t{bp_threshold}'], color=c)

    ax2.set_yticks(range(len(df_by_r_w_classes['r_id'])), df_by_r_w_classes['r_id'])
    for i, j in colors.items(): 
        ax2.barh(range(len(df_by_r_w_classes['r_id'])), df_by_r_w_classes[f'better_predicted_by_anyburl_t{bp_threshold}'],height=0,color=j,label=i)   
        
    ax2.legend()    
    
    plt_show()

In [ ]:
def show_better_predicted_triples_per_relation_class_table(df, dataset_name, use_difference_psi, bp_threshold):
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    relation_classes = get_relation_classes(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt')
    )    
    df_w_classes = df.join(relation_classes.set_index('r_id'), on='r_id')    
       
    triples_per_class = pd.DataFrame(columns=[symbolic_name, subsymbolic_name, 'Equal'])
    
    r_classes = ['1-1', '1-N', 'M-1', 'M-N']    
    for r_class in r_classes:
        if df_w_classes[df_w_classes['relationClass'] == r_class].shape[0] != 0:         
            ab = df_w_classes[(df_w_classes[f'better_predicted_by_anyburl_t{bp_threshold}'] > 0) & (df_w_classes['relationClass'] == r_class)].shape[0] / df_w_classes[df_w_classes['relationClass'] == r_class].shape[0]
            kge = df_w_classes[(df_w_classes[f'better_predicted_by_anyburl_t{bp_threshold}'] < 0) & (df_w_classes['relationClass'] == r_class)].shape[0] / df_w_classes[df_w_classes['relationClass'] == r_class].shape[0]
            equal = df_w_classes[(df_w_classes[f'better_predicted_by_anyburl_t{bp_threshold}'] == 0) & (df_w_classes['relationClass'] == r_class)].shape[0] / df_w_classes[df_w_classes['relationClass'] == r_class].shape[0]
        else: 
            ab = 0
            kge = 0
            equal = 0
        
        triples_per_class.loc[r_class] = [f'{(ab*100):.3f}%', f'{(kge*100):.3f}%', f'{(equal*100):.3f}%']
    
    fig, ax = plt.subplots(dpi=dpi)
    table = ax.table(colLabels=triples_per_class.columns, colWidths=[0.5] * len(triples_per_class.columns), rowLabels=triples_per_class.index, loc='center', cellText=triples_per_class.values)
        
    fig.suptitle(f'({get_title_key()}) Better predicted per Relation Class in %')
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')
    
    table.auto_set_font_size(True)

    plt_show(pad=1)

In [ ]:
def show_better_predicted_triples_per_relation_class_table_cleaned(df, dataset_name, use_difference_psi, bp_threshold):
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    relation_classes = get_relation_classes(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt')
    )    
    df_w_classes = df.join(relation_classes.set_index('r_id'), on='r_id')    
    df_w_classes['relationClassCleaned'] = df_w_classes['relationClass']
    df_w_classes.loc[(df_w_classes['relationClass'] == '1-N') & (df_w_classes['predicted_head'] == True), 'relationClassCleaned'] = 'M-1'
    df_w_classes.loc[(df_w_classes['relationClass'] == 'M-1') & (df_w_classes['predicted_head'] == True), 'relationClassCleaned'] = '1-N'
    
    triples_per_class = pd.DataFrame(columns=[symbolic_name, subsymbolic_name, 'Equal'])
    
    r_classes = ['1-1', '1-N', 'M-1', 'M-N']    
    for r_class in r_classes:
        if df_w_classes[df_w_classes['relationClassCleaned'] == r_class].shape[0] != 0:         
            ab = df_w_classes[(df_w_classes[f'better_predicted_by_anyburl_t{bp_threshold}'] > 0) & (df_w_classes['relationClassCleaned'] == r_class)].shape[0] / df_w_classes[df_w_classes['relationClassCleaned'] == r_class].shape[0]
            kge = df_w_classes[(df_w_classes[f'better_predicted_by_anyburl_t{bp_threshold}'] < 0) & (df_w_classes['relationClassCleaned'] == r_class)].shape[0] / df_w_classes[df_w_classes['relationClassCleaned'] == r_class].shape[0]
            equal = df_w_classes[(df_w_classes[f'better_predicted_by_anyburl_t{bp_threshold}'] == 0) & (df_w_classes['relationClassCleaned'] == r_class)].shape[0] / df_w_classes[df_w_classes['relationClassCleaned'] == r_class].shape[0]
        else: 
            ab = 0
            kge = 0
            equal = 0
        
        triples_per_class.loc[r_class] = [f'{(ab*100):.3f}%', f'{(kge*100):.3f}%', f'{(equal*100):.3f}%']
    
    fig, ax = plt.subplots(dpi=dpi)
    table = ax.table(colLabels=triples_per_class.columns, colWidths=[0.5] * len(triples_per_class.columns), rowLabels=triples_per_class.index, loc='center', cellText=triples_per_class.values)
        
    fig.suptitle(f'({get_title_key()}) Better predicted per Relation Class in % (cleaned)')
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')
    
    table.auto_set_font_size(True)

    plt_show(pad=1)

# Better Predicted by AnyBURL per Entity with Entity Frequency

In [ ]:
def show_better_predicted_per_head_with_frequency(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 'h_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    ht_freq = get_entity_frequency_in_training_data(dataset_name)
    
    df_by_h = group_df_by_h_id(df)
    df_by_h_w_freq = ht_freq.join(df_by_h, on='h_id').sort_values('freq')
    df_by_h_w_freq.reset_index(inplace=True)
    df_by_h_w_freq = df_by_h_w_freq.dropna()
    df_by_h_w_freq = df_by_h_w_freq.sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    
    cmap = plt.cm.viridis
    norm = matplotlib.colors.Normalize(vmin=df_by_h_w_freq['norm_freq'].min(), vmax=df_by_h_w_freq['norm_freq'].nlargest(1000).values[100])

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.bar(range(len(df_by_h_w_freq['h_id'])), df_by_h_w_freq[f'better_predicted_by_anyburl_t{bp_threshold}'], color=cmap(norm(df_by_h_w_freq['norm_freq'].values)))
    
    ax.set_xticks(range(len(df_by_h_w_freq['h_id'])), df_by_h_w_freq['h_id'])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Normalized Entity Frequency')

    plt.title(f'({get_title_key()}) Better predicted by AnyBURL per Head with Entity Frequency in Trainings Data')
    plt.xlabel('Entity Id')
    plt.ylabel('Better predicted by AnyBURL')
    plt.ylim(-1,1)

    plt_show()

In [ ]:
def show_better_predicted_per_tail_with_frequency(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 't_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    ht_freq = get_entity_frequency_in_training_data(dataset_name)
    
    df_by_t = group_df_by_h_id(df)
    df_by_t_w_freq = ht_freq.join(df_by_t, on='t_id').sort_values('freq')
    df_by_t_w_freq.reset_index(inplace=True)
    df_by_t_w_freq = df_by_t_w_freq.dropna()
    df_by_t_w_freq = df_by_t_w_freq.sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    
    cmap = plt.cm.viridis
    norm = matplotlib.colors.Normalize(vmin=df_by_t_w_freq['norm_freq'].min(), vmax=df_by_t_w_freq['norm_freq'].nlargest(1000).values[100])

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.bar(range(len(df_by_t_w_freq['h_id'])), df_by_t_w_freq[f'better_predicted_by_anyburl_t{bp_threshold}'], color=cmap(norm(df_by_t_w_freq['norm_freq'].values)))
    
    ax.set_xticks(range(len(df_by_t_w_freq['t_id'])), df_by_t_w_freq['t_id'])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Normalized Entity Frequency')

    plt.title(f'({get_title_key()}) Better predicted by AnyBURL per Tail with Entity Frequency in Trainings Data')
    plt.xlabel('Entity Id')
    plt.ylabel('Better predicted by AnyBURL')
    plt.ylim(-1,1)

    plt_show()

# Better Predicted by AnyBURL by similar Triple in Trainings Data in Steps

In [ ]:
def show_better_predicted_by_similar_triple_exists_steps(df, dataset_name, use_difference_psi, bp_threshold):
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
   
    df_w_similar_triple_count = get_similar_triples_in_trainings_data(df, dataset_name)
    df_w_similar_triple_count['has_similar_triples'] = df_w_similar_triple_count['similar_triples'] > 0
    
    df_w_similar_triple_count['similar_bin'] = ''    
    df_w_similar_triple_count.loc[df_w_similar_triple_count['similar_triples'] == 0, 'similar_bin'] = '0'
    df_w_similar_triple_count.loc[(df_w_similar_triple_count['similar_triples'] > 0) & (df_w_similar_triple_count['similar_triples'] <= 10), 'similar_bin'] = '01 - 10'
    df_w_similar_triple_count.loc[(df_w_similar_triple_count['similar_triples'] > 10) & (df_w_similar_triple_count['similar_triples'] <= 20),'similar_bin'] = '11 - 20'
    df_w_similar_triple_count.loc[(df_w_similar_triple_count['similar_triples'] > 20) & (df_w_similar_triple_count['similar_triples'] <= 30),'similar_bin'] = '21 - 30'
    df_w_similar_triple_count.loc[(df_w_similar_triple_count['similar_triples'] > 30) & (df_w_similar_triple_count['similar_triples'] <= 40),'similar_bin'] = '31 - 40'
    df_w_similar_triple_count.loc[(df_w_similar_triple_count['similar_triples'] > 40) & (df_w_similar_triple_count['similar_triples'] <= 50),'similar_bin'] = '41 - 50'
    df_w_similar_triple_count.loc[(df_w_similar_triple_count['similar_triples'] > 50) & (df_w_similar_triple_count['similar_triples'] <= 60),'similar_bin'] = '51 - 60'
    df_w_similar_triple_count.loc[(df_w_similar_triple_count['similar_triples'] > 60) & (df_w_similar_triple_count['similar_triples'] <= 70),'similar_bin'] = '61 - 70'
    df_w_similar_triple_count.loc[(df_w_similar_triple_count['similar_triples'] > 70) & (df_w_similar_triple_count['similar_triples'] <= 80),'similar_bin'] = '71 - 80'
    df_w_similar_triple_count.loc[(df_w_similar_triple_count['similar_triples'] > 80) & (df_w_similar_triple_count['similar_triples'] <= 90),'similar_bin'] = '81 - 90'
    df_w_similar_triple_count.loc[(df_w_similar_triple_count['similar_triples'] > 90) & (df_w_similar_triple_count['similar_triples'] <= 100),'similar_bin'] = '91 - 100'
    df_w_similar_triple_count.loc[df_w_similar_triple_count['similar_triples'] > 100, 'similar_bin'] = '> 100'
    
    df_by_similar_bin = df_w_similar_triple_count[['similar_bin', f'better_predicted_by_anyburl_t{bp_threshold}']].groupby('similar_bin').mean().reset_index()
        
    plt.figure(figsize=figsize, dpi=dpi)
    plt.bar(df_by_similar_bin['similar_bin'], df_by_similar_bin[f'better_predicted_by_anyburl_t{bp_threshold}'])
    
    plt.xticks(df_by_similar_bin['similar_bin'])
    
    plt.ylabel('Better predicted by AnyBURL')
    plt.xlabel('Similar Triples existing in Trainings Data')
    plt.title(f'({get_title_key()}) Better Predicted by AnyBURL per Similar Triple in Trainings Data')
    plt.ylim(-1,1)
    plt_show()

# Better Predicted by AnyBURL by similar Triple in Trainings Data

In [ ]:
def show_better_predicted_by_similar_triple_exists(df, dataset_name, use_difference_psi, bp_threshold):
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
   
    df_w_similar_triple_count = get_similar_triples_in_trainings_data(df, dataset_name)
    df_w_similar_triple_count['has_similar_triples'] = df_w_similar_triple_count['similar_triples'] > 0
    df_by_has_similar = df_w_similar_triple_count[['has_similar_triples', f'better_predicted_by_anyburl_t{bp_threshold}']].groupby('has_similar_triples').mean().reset_index()
    
    plt.figure(figsize=figsize, dpi=dpi)
    plt.bar(df_by_has_similar['has_similar_triples'], df_by_has_similar[f'better_predicted_by_anyburl_t{bp_threshold}'])
    
    plt.xticks(df_by_has_similar['has_similar_triples'])
    
    plt.ylabel('Better predicted by AnyBURL')
    plt.xlabel('Similar Triples existing in Trainings Data')
    plt.title(f'({get_title_key()}) Better Predicted by AnyBURL per Similar Triple in Trainings Data')
    plt.ylim(-1,1)
    plt_show()

# Better Predicted by AnyBURL per Entity Frequency Bin

In [ ]:
def show_better_predicted_per_head_frequency_bin(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 'h_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    ht_freq = get_entity_frequency_in_training_data(dataset_name)[['h_id', 'freq_h', 'norm_freq_h']]
    
    df_w_freq = df.join(ht_freq.set_index('h_id'), on='h_id', how='left')
    df_w_freq['bin'] = pd.cut(df_w_freq['norm_freq_h'], 10, retbins=True)[0]
    df_w_freq_by_bin = df_w_freq[['bin', f'better_predicted_by_anyburl_t{bp_threshold}']].groupby('bin').mean()
    df_w_freq_by_bin.reset_index(inplace=True)
    
    plt.figure(figsize=figsize, dpi=dpi)
    plt.barh(range(len(df_w_freq_by_bin['bin'])),  df_w_freq_by_bin[f'better_predicted_by_anyburl_t{bp_threshold}'])
    
    plt.yticks(range(len(df_w_freq_by_bin['bin'])), df_w_freq_by_bin['bin'])
    plt.xlabel('Better predicted by AnyBURL')
    plt.ylabel('Normalized Entity Frequency Bin')
    plt.title(f'({get_title_key()}) Better Predicted by AnyBURL per Normalized Head Frequency Bin')
    plt.xlim(-1,1)
    
    plt_show()

In [ ]:
def show_better_predicted_per_tail_frequency_bin(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 't_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    ht_freq = get_entity_frequency_in_training_data(dataset_name)[['t_id', 'freq_t', 'norm_freq_t']]
    
    df_w_freq = df.join(ht_freq.set_index('t_id'), on='t_id', how='left')
    df_w_freq['bin'] = pd.cut(df_w_freq['norm_freq_t'], 10, retbins=True)[0]
    df_w_freq_by_bin = df_w_freq[['bin', f'better_predicted_by_anyburl_t{bp_threshold}']].groupby('bin').mean()
    df_w_freq_by_bin.reset_index(inplace=True)
    
    plt.figure(figsize=figsize, dpi=dpi)
    plt.barh(range(len(df_w_freq_by_bin['bin'])),  df_w_freq_by_bin[f'better_predicted_by_anyburl_t{bp_threshold}'])
    
    plt.yticks(range(len(df_w_freq_by_bin['bin'])), df_w_freq_by_bin['bin'])
    plt.xlabel('Better predicted by AnyBURL')
    plt.ylabel('Normalized Entity Frequency Bin')
    plt.title(f'({get_title_key()}) Better Predicted by AnyBURL per Normalized Tail Frequency Bin')
    plt.xlim(-1,1)
    
    plt_show()

In [ ]:
def show_better_predicted_per_combined_entity_frequency_bin(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 't_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    t_freq = get_entity_frequency_in_training_data(dataset_name)[['t_id', 'freq_t', 'norm_freq_t']]
    h_freq = get_entity_frequency_in_training_data(dataset_name)[['h_id', 'freq_h', 'norm_freq_h']]
    r_freq = get_relation_frequency_in_training_data(dataset_name)
    
    df_w_freq = df.join(t_freq.set_index('t_id'), on='t_id', how='left').join(h_freq.set_index('h_id'), on='h_id', how='left').join(r_freq.set_index('r_id'), on='r_id', how='left')
    df_w_freq['norm_freq'] =  df_w_freq['norm_freq_h'] +df_w_freq['norm_freq']+df_w_freq['norm_freq_t']
    df_w_freq['norm_freq'] =  (df_w_freq['norm_freq'] - df_w_freq['norm_freq'].min())/(df_w_freq['norm_freq'].max()-df_w_freq['norm_freq'].min())
    df_w_freq['bin'] = pd.cut(df_w_freq['norm_freq'], 10, retbins=True)[0]
    df_w_freq_by_bin = df_w_freq[['bin', f'better_predicted_by_anyburl_t{bp_threshold}']].groupby('bin').mean()
    df_w_freq_by_bin.reset_index(inplace=True)
    
    plt.figure(figsize=figsize, dpi=dpi)
    plt.barh(range(len(df_w_freq_by_bin['bin'])),  df_w_freq_by_bin[f'better_predicted_by_anyburl_t{bp_threshold}'])
    
    plt.yticks(range(len(df_w_freq_by_bin['bin'])), df_w_freq_by_bin['bin'])
    plt.xlabel('Better predicted by AnyBURL')
    plt.ylabel('Normalized Entity Frequency Bin')
    plt.title(f'({get_title_key()}) Better Predicted by AnyBURL per Combined Normalized Entity Frequency Bin')
    plt.xlim(-1,1)
    
    plt_show()

In [7]:
def show_better_predicted_per_question_entity_frequency_bin(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 't_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    t_freq = get_entity_frequency_in_training_data(dataset_name)[['t_id', 'norm_freq_t']]
    h_freq = get_entity_frequency_in_training_data(dataset_name)[['h_id', 'norm_freq_h']]
    
    df_w_freq = df.join(t_freq.set_index('t_id'), on='t_id', how='left').join(h_freq.set_index('h_id'), on='h_id', how='left')
    df_w_freq.loc[(df_w_freq['predicted_head'] == True),'norm_freq'] = df_w_freq.loc[(df_w_freq['predicted_head'] == True),'norm_freq_t']
    df_w_freq.loc[(df_w_freq['predicted_head'] == False),'norm_freq'] = df_w_freq.loc[(df_w_freq['predicted_head'] == False),'norm_freq_h']
    
    df_w_freq['bin'] = pd.cut(df_w_freq['norm_freq'], 10, retbins=True)[0]
    df_w_freq_by_bin = df_w_freq[['bin', f'better_predicted_by_anyburl_t{bp_threshold}']].groupby('bin').mean()
    df_w_freq_by_bin.reset_index(inplace=True)
    
    plt.figure(figsize=figsize, dpi=dpi)
    plt.barh(range(len(df_w_freq_by_bin['bin'])),  df_w_freq_by_bin[f'better_predicted_by_anyburl_t{bp_threshold}'])
    
    plt.yticks(range(len(df_w_freq_by_bin['bin'])), df_w_freq_by_bin['bin'])
    plt.xlabel('Better predicted by AnyBURL')
    plt.ylabel('Normalized Entity Frequency Bin')
    plt.title(f'({get_title_key()}) Better Predicted by AnyBURL per Normalized Question Entity Frequency Bin')
    plt.xlim(-1,1)
    
    plt_show()

In [ ]:
def show_better_predicted_per_answer_entity_frequency_bin(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 't_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    t_freq = get_entity_frequency_in_training_data(dataset_name)[['t_id', 'norm_freq_t']]
    h_freq = get_entity_frequency_in_training_data(dataset_name)[['h_id', 'norm_freq_h']]
    
    df_w_freq = df.join(t_freq.set_index('t_id'), on='t_id', how='left').join(h_freq.set_index('h_id'), on='h_id', how='left')
    df_w_freq.loc[(df_w_freq['predicted_head'] == False),'norm_freq'] = df_w_freq.loc[(df_w_freq['predicted_head'] == False),'norm_freq_t']
    df_w_freq.loc[(df_w_freq['predicted_head'] == True),'norm_freq'] = df_w_freq.loc[(df_w_freq['predicted_head'] == True),'norm_freq_h']
    
    df_w_freq['bin'] = pd.cut(df_w_freq['norm_freq'], 10, retbins=True)[0]
    df_w_freq_by_bin = df_w_freq[['bin', f'better_predicted_by_anyburl_t{bp_threshold}']].groupby('bin').mean()
    df_w_freq_by_bin.reset_index(inplace=True)
    
    plt.figure(figsize=figsize, dpi=dpi)
    plt.barh(range(len(df_w_freq_by_bin['bin'])),  df_w_freq_by_bin[f'better_predicted_by_anyburl_t{bp_threshold}'])
    
    plt.yticks(range(len(df_w_freq_by_bin['bin'])), df_w_freq_by_bin['bin'])
    plt.xlabel('Better predicted by AnyBURL')
    plt.ylabel('Normalized Entity Frequency Bin')
    plt.title(f'({get_title_key()}) Better Predicted by AnyBURL per Normalized Answer Entity Frequency Bin')
    plt.xlim(-1,1)
    
    plt_show()

# Correlation Entity Frequency Difference Psi

In [ ]:
def show_correlation_between_entity_frequency_and_difference_psi(df, dataset_name):
    """
    expects a df with 'h_id', 't_id' 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    ht_freq = get_entity_frequency_in_training_data(dataset_name)[['h_id', 'norm_freq', 'norm_freq_h', 'norm_freq_t']]    
    df_w_freq = df.join(ht_freq.set_index('h_id'), on='h_id', how='left')
            
    
    print('Pearson correlation between Entity Frequency and Difference Psi')
    print(f"for triples = {df_w_freq['norm_freq'].corr(df_w_freq['difference_psi'])}")
    print(f"for heads = {df_w_freq['norm_freq_h'].corr(df_w_freq['difference_psi'])}")
    print(f"for tails = {df_w_freq['norm_freq_t'].corr(df_w_freq['difference_psi'])}")
    pdf.attach_note(f"Pearson correlation between Entity Frequency and Difference Psi: for triples = {df_w_freq['norm_freq'].corr(df_w_freq['difference_psi'])}; for heads = {df_w_freq['norm_freq_h'].corr(df_w_freq['difference_psi'])}; for tails = {df_w_freq['norm_freq_t'].corr(df_w_freq['difference_psi'])}")

# Better Predicted by with Entity Similarity Score

In [ ]:
def show_better_predicted_per_head_with_sim_score(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 'h_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    ht_sim_score = get_entity_similarity_scores(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt')
    )
    
    df_by_h = group_df_by_h_id(df)
    df_by_h_w_sim = ht_sim_score.join(df_by_h, on='h_id').sort_values('norm_entity_sim_score')
    df_by_h_w_sim.reset_index(inplace=True)
    df_by_h_w_sim = df_by_h_w_sim.dropna()
    df_by_h_w_sim = df_by_h_w_sim.sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    
    cmap = plt.cm.viridis
    norm = matplotlib.colors.Normalize(vmin=df_by_h_w_sim['norm_entity_sim_score'].min(), vmax=df_by_h_w_sim['norm_entity_sim_score'].nlargest(100).values[0])

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.bar(range(len(df_by_h_w_sim['h_id'])), df_by_h_w_sim[f'better_predicted_by_anyburl_t{bp_threshold}'], color=cmap(norm(df_by_h_w_sim['norm_entity_sim_score'].values)))
    
    ax.set_xticks(range(len(df_by_h_w_sim['h_id'])), df_by_h_w_sim['h_id'])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Normalized Entity Similarity Score')

    plt.title(f'({get_title_key()}) Better predicted by AnyBURL per Head with Entity Similarity Score')
    plt.xlabel('Entity Id')
    plt.ylabel('Better predicted by AnyBURL')
    plt.ylim(-1,1)

    plt_show()

In [ ]:
def show_better_predicted_per_head_with_sim_score_swaped(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 'h_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    ht_sim_score = get_entity_similarity_scores(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt')
    )
    
    df_by_h = group_df_by_h_id(df)
    df_by_h_w_sim = ht_sim_score.join(df_by_h, on='h_id').sort_values('norm_entity_sim_score')
    df_by_h_w_sim.reset_index(inplace=True)
    df_by_h_w_sim = df_by_h_w_sim.dropna()
    df_by_h_w_sim = df_by_h_w_sim.sort_values('norm_entity_sim_score')
    
    cmap = plt.cm.RdYlGn
    norm = matplotlib.colors.Normalize(vmin=df_by_h_w_sim[f'better_predicted_by_anyburl_t{bp_threshold}'].min(), vmax=df_by_h_w_sim[f'better_predicted_by_anyburl_t{bp_threshold}'].nlargest(100).values[0])

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.bar(range(len(df_by_h_w_sim['h_id'])), df_by_h_w_sim['norm_entity_sim_score'], color=cmap(norm(df_by_h_w_sim[f'better_predicted_by_anyburl_t{bp_threshold}'].values)))
    
    ax.set_xticks(range(len(df_by_h_w_sim['h_id'])), df_by_h_w_sim['h_id'])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Better predicted by AnyBURL')

    plt.title(f'({get_title_key()}) Better predicted by AnyBURL per Head with Entity Similarity Score')
    plt.xlabel('Entity Id')
    plt.ylabel('Normalized Entity Similarity Score')
    plt.ylim(0,1)

    plt_show()

In [ ]:
def show_better_predicted_per_tail_with_sim_score(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 't_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    ht_sim_score = get_entity_similarity_scores(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt')
    )
    
    df_by_h = group_df_by_t_id(df)
    df_by_h_w_sim = ht_sim_score.join(df_by_h, on='t_id').sort_values('norm_entity_sim_score')
    df_by_h_w_sim.reset_index(inplace=True)
    df_by_h_w_sim = df_by_h_w_sim.dropna()
    df_by_h_w_sim = df_by_h_w_sim.sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    
    cmap = plt.cm.viridis
    norm = matplotlib.colors.Normalize(vmin=df_by_h_w_sim['norm_entity_sim_score'].min(), vmax=df_by_h_w_sim['norm_entity_sim_score'].nlargest(100).values[0])

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.bar(range(len(df_by_h_w_sim['t_id'])), df_by_h_w_sim[f'better_predicted_by_anyburl_t{bp_threshold}'], color=cmap(norm(df_by_h_w_sim['norm_entity_sim_score'].values)))
    
    ax.set_xticks(range(len(df_by_h_w_sim['t_id'])), df_by_h_w_sim['t_id'])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Normalized Entity Similarity Score')

    plt.title(f'({get_title_key()}) Better predicted by AnyBURL per Tail with Entity Similarity Score')
    plt.xlabel('Entity Id')
    plt.ylabel('Better predicted by AnyBURL')
    plt.ylim(-1,1)

    plt_show()

In [ ]:
def show_better_predicted_per_tail_with_sim_score_swaped(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 't_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    ht_sim_score = get_entity_similarity_scores(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt')
    )
    
    df_by_h = group_df_by_t_id(df)
    df_by_h_w_sim = ht_sim_score.join(df_by_h, on='t_id').sort_values('norm_entity_sim_score')
    df_by_h_w_sim.reset_index(inplace=True)
    df_by_h_w_sim = df_by_h_w_sim.dropna()
    df_by_h_w_sim = df_by_h_w_sim.sort_values('norm_entity_sim_score')
    
    cmap = plt.cm.RdYlGn
    norm = matplotlib.colors.Normalize(vmin=df_by_h_w_sim[f'better_predicted_by_anyburl_t{bp_threshold}'].min(), vmax=df_by_h_w_sim[f'better_predicted_by_anyburl_t{bp_threshold}'].nlargest(100).values[0])

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.bar(range(len(df_by_h_w_sim['t_id'])), df_by_h_w_sim['norm_entity_sim_score'], color=cmap(norm(df_by_h_w_sim[f'better_predicted_by_anyburl_t{bp_threshold}'].values)))
    
    ax.set_xticks(range(len(df_by_h_w_sim['t_id'])), df_by_h_w_sim['t_id'])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Better predicted by AnyBURL')

    plt.title(f'({get_title_key()}) Better predicted by AnyBURL per Tail with Entity Similarity Score')
    plt.xlabel('Entity Id')
    plt.ylabel('Normalized Entity Similarity Score')
    plt.ylim(0,1)

    plt_show()

In [ ]:
def show_better_predicted_per_head_question_answer_entity_with_sim_score(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 'h_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    ht_sim_score = get_entity_similarity_scores(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt')
    )[['h_id', 'norm_entity_sim_score']]
    
    df_ph = group_df_by_h_id(df[df['predicted_head'] == True])
    df_pt = group_df_by_h_id(df[df['predicted_head'] == False])
    
    df_ph_by_h_w_sim = ht_sim_score.join(df_ph, on='h_id').reset_index().dropna().sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    df_pt_by_h_w_sim = ht_sim_score.join(df_pt, on='h_id').reset_index().dropna().sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
        
    cmap = plt.cm.viridis
    norm = matplotlib.colors.Normalize(vmin=df_ph_by_h_w_sim['norm_entity_sim_score'].min(), vmax=df_ph_by_h_w_sim['norm_entity_sim_score'].nlargest(100).values[0])

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize, dpi=dpi)
    ax1.bar(range(len(df_pt_by_h_w_sim['h_id'])), df_pt_by_h_w_sim[f'better_predicted_by_anyburl_t{bp_threshold}'], color=cmap(norm(df_pt_by_h_w_sim['norm_entity_sim_score'].values)))
    ax1.set_xticks(range(len(df_pt_by_h_w_sim['h_id'])), df_pt_by_h_w_sim['h_id'])
    ax1.set_xlabel('Entity Id')
    ax1.set_ylabel('Better predicted by AnyBURL')
    ax1.set_ylim(-1,1)
    ax1.set_title('Similarity based on Question Entity')    
    
    ax2.bar(range(len(df_ph_by_h_w_sim['h_id'])), df_ph_by_h_w_sim[f'better_predicted_by_anyburl_t{bp_threshold}'], color=cmap(norm(df_ph_by_h_w_sim['norm_entity_sim_score'].values)))
    ax2.set_xticks(range(len(df_ph_by_h_w_sim['h_id'])), df_ph_by_h_w_sim['h_id'])
    ax2.set_xlabel('Entity Id')
    ax2.set_ylabel('Better predicted by AnyBURL')
    ax2.set_ylim(-1,1)
    ax2.set_title('Similarity based on Answer Entity')
    
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Normalized Entity Similarity Score')

    fig.suptitle(f'({get_title_key()}) Better predicted by AnyBURL per Head with Entity Similarity Score')

    plt_show()

In [ ]:
def show_better_predicted_per_tail_question_answer_entity_with_sim_score(df, dataset_name, use_difference_psi, bp_threshold):
    """
    expects a df with 't_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    ht_sim_score = get_entity_similarity_scores(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt')
    )[['t_id', 'norm_entity_sim_score']]
    
    df_ph = group_df_by_t_id(df[df['predicted_head'] == True])
    df_pt = group_df_by_t_id(df[df['predicted_head'] == False])
    
    df_ph_by_h_w_sim = ht_sim_score.join(df_ph, on='t_id').reset_index().dropna().sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    df_pt_by_h_w_sim = ht_sim_score.join(df_pt, on='t_id').reset_index().dropna().sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
        
    cmap = plt.cm.viridis
    norm = matplotlib.colors.Normalize(vmin=df_ph_by_h_w_sim['norm_entity_sim_score'].min(), vmax=df_ph_by_h_w_sim['norm_entity_sim_score'].nlargest(100).values[0])

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize, dpi=dpi)
    ax2.bar(range(len(df_pt_by_h_w_sim['t_id'])), df_pt_by_h_w_sim[f'better_predicted_by_anyburl_t{bp_threshold}'], color=cmap(norm(df_pt_by_h_w_sim['norm_entity_sim_score'].values)))
    ax2.set_xticks(range(len(df_pt_by_h_w_sim['t_id'])), df_pt_by_h_w_sim['t_id'])
    ax2.set_xlabel('Entity Id')
    ax2.set_ylabel('Better predicted by AnyBURL')
    ax2.set_ylim(-1,1)
    ax2.set_title('Similarity based on Answer Entity')   
    
    ax1.bar(range(len(df_ph_by_h_w_sim['t_id'])), df_ph_by_h_w_sim[f'better_predicted_by_anyburl_t{bp_threshold}'], color=cmap(norm(df_ph_by_h_w_sim['norm_entity_sim_score'].values)))
    ax1.set_xticks(range(len(df_ph_by_h_w_sim['t_id'])), df_ph_by_h_w_sim['t_id'])
    ax1.set_xlabel('Entity Id')
    ax1.set_ylabel('Better predicted by AnyBURL')
    ax1.set_ylim(-1,1)
    ax1.set_title('Similarity based on Question Entity') 
    
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Normalized Entity Similarity Score')

    fig.suptitle(f'({get_title_key()}) Better predicted by AnyBURL per Tail with Entity Similarity Score')

    plt_show()

# Better Predicted by with Entity Clusters

In [ ]:
def show_better_predicted_per_head_cluster(df, dataset_name, use_difference_psi, bp_threshold, n):
    """
    expects a df with 'h_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    entity_cluster = get_entity_clusters(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt'), n
    )[['h_id', 'cluster_id', 'freq', 'norm_freq']]
    
    df_w_cluster = df.join(entity_cluster.set_index('h_id'), on='h_id')
    df_by_cluster = df_w_cluster[['cluster_id', f'better_predicted_by_anyburl_t{bp_threshold}', 'freq', 'norm_freq']].groupby('cluster_id').mean()
    df_by_cluster.reset_index(inplace=True)
    df_by_cluster = df_by_cluster.sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    
    cmap = plt.cm.viridis
    norm = matplotlib.colors.Normalize(vmin=df_by_cluster['norm_freq'].min(), vmax=df_by_cluster['norm_freq'].nlargest(100).values[0])

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.bar(range(len(df_by_cluster['cluster_id'])), df_by_cluster[f'better_predicted_by_anyburl_t{bp_threshold}'], color=cmap(norm(df_by_cluster['norm_freq'].values)))
    
    ax.set_xticks(range(len(df_by_cluster['cluster_id'])), df_by_cluster['cluster_id'])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Normalized Entity Cluster Size')

    plt.title(f'({get_title_key()}) Better predicted by AnyBURL per Head Cluster with Normalized Entity Cluster Size (n={n})')
    plt.xlabel('Cluster Id')
    plt.ylabel('Better predicted by AnyBURL')
    plt.ylim(-1,1)

    plt_show()

In [ ]:
def show_better_predicted_per_head_cluster_swaped(df, dataset_name, use_difference_psi, bp_threshold, n):
    """
    expects a df with 'h_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    entity_cluster = get_entity_clusters(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt'), n
    )[['h_id', 'cluster_id', 'freq', 'norm_freq']]
    
    df_w_cluster = df.join(entity_cluster.set_index('h_id'), on='h_id')
    df_by_cluster = df_w_cluster[['cluster_id', f'better_predicted_by_anyburl_t{bp_threshold}', 'freq', 'norm_freq']].groupby('cluster_id').mean()
    df_by_cluster.reset_index(inplace=True)
    df_by_cluster = df_by_cluster.sort_values(f'norm_freq')
    
    cmap = plt.cm.RdYlGn
    norm = matplotlib.colors.Normalize(vmin=df_by_cluster[f'better_predicted_by_anyburl_t{bp_threshold}'].min(), vmax=df_by_cluster[f'better_predicted_by_anyburl_t{bp_threshold}'].nlargest(100).values[0])

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.bar(range(len(df_by_cluster['cluster_id'])), df_by_cluster[f'norm_freq'], color=cmap(norm(df_by_cluster[f'better_predicted_by_anyburl_t{bp_threshold}'].values)))
    
    ax.set_xticks(range(len(df_by_cluster['cluster_id'])), df_by_cluster['cluster_id'])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Normalized Entity Cluster Size')

    plt.title(f'({get_title_key()}) Better predicted by AnyBURL per Head Cluster with Normalized Entity Cluster Size (n={n})')
    plt.xlabel('Cluster Id')
    plt.ylabel('Better predicted by AnyBURL')
    plt.ylim(0,1)

    plt_show()

In [ ]:
def show_better_predicted_per_tail_cluster(df, dataset_name, use_difference_psi, bp_threshold, n):
    """
    expects a df with 't_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    entity_cluster = get_entity_clusters(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt'), n
    )[['t_id', 'cluster_id', 'freq', 'norm_freq']]
    
    df_w_cluster = df.join(entity_cluster.set_index('t_id'), on='t_id')
    df_by_cluster = df_w_cluster[['cluster_id', f'better_predicted_by_anyburl_t{bp_threshold}', 'freq', 'norm_freq']].groupby('cluster_id').mean()
    df_by_cluster.reset_index(inplace=True)
    df_by_cluster = df_by_cluster.sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    
    cmap = plt.cm.viridis
    norm = matplotlib.colors.Normalize(vmin=df_by_cluster['norm_freq'].min(), vmax=df_by_cluster['norm_freq'].nlargest(100).values[0])

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.bar(range(len(df_by_cluster['cluster_id'])), df_by_cluster[f'better_predicted_by_anyburl_t{bp_threshold}'], color=cmap(norm(df_by_cluster['norm_freq'].values)))
    
    ax.set_xticks(range(len(df_by_cluster['cluster_id'])), df_by_cluster['cluster_id'])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Normalized Entity Cluster Size')

    plt.title(f'({get_title_key()}) Better predicted by AnyBURL per Tail Cluster with Normalized Entity Cluster Size (n={n})')
    plt.xlabel('Cluster Id')
    plt.ylabel('Better predicted by AnyBURL')
    plt.ylim(-1,1)

    plt_show()   

In [ ]:
def show_better_predicted_per_tail_cluster_swaped(df, dataset_name, use_difference_psi, bp_threshold, n):
    """
    expects a df with 't_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    entity_cluster = get_entity_clusters(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt'), n
    )[['t_id', 'cluster_id', 'freq', 'norm_freq']]
    
    df_w_cluster = df.join(entity_cluster.set_index('t_id'), on='t_id')
    df_by_cluster = df_w_cluster[['cluster_id', f'better_predicted_by_anyburl_t{bp_threshold}', 'freq', 'norm_freq']].groupby('cluster_id').mean()
    df_by_cluster.reset_index(inplace=True)
    df_by_cluster = df_by_cluster.sort_values(f'norm_freq')
    
    cmap = plt.cm.RdYlGn
    norm = matplotlib.colors.Normalize(vmin=df_by_cluster[f'better_predicted_by_anyburl_t{bp_threshold}'].min(), vmax=df_by_cluster[f'better_predicted_by_anyburl_t{bp_threshold}'].nlargest(100).values[0])

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.bar(range(len(df_by_cluster['cluster_id'])), df_by_cluster[f'norm_freq'], color=cmap(norm(df_by_cluster[f'better_predicted_by_anyburl_t{bp_threshold}'].values)))
    
    ax.set_xticks(range(len(df_by_cluster['cluster_id'])), df_by_cluster['cluster_id'])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Normalized Entity Cluster Size')

    plt.title(f'({get_title_key()}) Better predicted by AnyBURL per Tail Cluster with Normalized Entity Cluster Size (n={n})')
    plt.xlabel('Cluster Id')
    plt.ylabel('Better predicted by AnyBURL')
    plt.ylim(0,1)

    plt_show()

In [ ]:
def show_better_predicted_per_tail_cluster(df, dataset_name, use_difference_psi, bp_threshold, n):
    """
    expects a df with 't_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    entity_cluster = get_entity_clusters(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt'), n
    )[['t_id', 'cluster_id', 'freq', 'norm_freq']]
    
    df_w_cluster = df.join(entity_cluster.set_index('t_id'), on='t_id')
    df_by_cluster = df_w_cluster[['cluster_id', f'better_predicted_by_anyburl_t{bp_threshold}', 'freq', 'norm_freq']].groupby('cluster_id').mean()
    df_by_cluster.reset_index(inplace=True)
    df_by_cluster = df_by_cluster.sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    
    cmap = plt.cm.viridis
    norm = matplotlib.colors.Normalize(vmin=df_by_cluster['norm_freq'].min(), vmax=df_by_cluster['norm_freq'].nlargest(100).values[0])

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.bar(range(len(df_by_cluster['cluster_id'])), df_by_cluster[f'better_predicted_by_anyburl_t{bp_threshold}'], color=cmap(norm(df_by_cluster['norm_freq'].values)))
    
    ax.set_xticks(range(len(df_by_cluster['cluster_id'])), df_by_cluster['cluster_id'])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Normalized Entity Cluster Size')

    plt.title(f'({get_title_key()}) Better predicted by AnyBURL per Tail Cluster with Normalized Entity Cluster Size (n={n})')
    plt.xlabel('Cluster Id')
    plt.ylabel('Better predicted by AnyBURL')
    plt.ylim(-1,1)

    plt_show()   

In [ ]:
def show_better_predicted_per_question_cluster(df, dataset_name, use_difference_psi, bp_threshold, n):
    """
    expects a df with 't_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    entity_cluster = get_entity_clusters(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt'), n
    )[['h_id', 't_id', 'cluster_id', 'freq', 'norm_freq']]
    
    entity_cluster_h = entity_cluster[['h_id', 'cluster_id', 'freq', 'norm_freq']].rename(columns={'cluster_id': 'cluster_id_h','freq':'freq_h', 'norm_freq':'norm_freq_h'})
    entity_cluster_t = entity_cluster[['t_id', 'cluster_id', 'freq', 'norm_freq']].rename(columns={'cluster_id': 'cluster_id_t','freq':'freq_t', 'norm_freq':'norm_freq_t'})
    
    df_w_cluster = df.join(entity_cluster_t.set_index('t_id')).join(entity_cluster_h.set_index('h_id'))
    
    df_w_cluster.loc[(df_w_cluster['predicted_head'] == True),'cluster_id'] = df_w_cluster.loc[(df_w_cluster['predicted_head'] == True),'cluster_id_t']
    df_w_cluster.loc[(df_w_cluster['predicted_head'] == False),'cluster_id'] = df_w_cluster.loc[(df_w_cluster['predicted_head'] == False),'cluster_id_h']
    
    df_w_cluster.loc[(df_w_cluster['predicted_head'] == True),'freq'] = df_w_cluster.loc[(df_w_cluster['predicted_head'] == True),'freq_t']
    df_w_cluster.loc[(df_w_cluster['predicted_head'] == False),'freq'] = df_w_cluster.loc[(df_w_cluster['predicted_head'] == False),'freq_h']
    
    df_w_cluster.loc[(df_w_cluster['predicted_head'] == True),'norm_freq'] = df_w_cluster.loc[(df_w_cluster['predicted_head'] == True),'norm_freq_t']
    df_w_cluster.loc[(df_w_cluster['predicted_head'] == False),'norm_freq'] = df_w_cluster.loc[(df_w_cluster['predicted_head'] == False),'norm_freq_h']
    
    
    df_by_cluster = df_w_cluster[['cluster_id', f'better_predicted_by_anyburl_t{bp_threshold}', 'freq', 'norm_freq']].groupby('cluster_id').mean()
    df_by_cluster.reset_index(inplace=True)
    df_by_cluster = df_by_cluster.sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    
    cmap = plt.cm.viridis
    norm = matplotlib.colors.Normalize(vmin=df_by_cluster['norm_freq'].min(), vmax=df_by_cluster['norm_freq'].nlargest(100).values[0])

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.bar(range(len(df_by_cluster['cluster_id'])), df_by_cluster[f'better_predicted_by_anyburl_t{bp_threshold}'], color=cmap(norm(df_by_cluster['norm_freq'].values)))
    
    ax.set_xticks(range(len(df_by_cluster['cluster_id'])), df_by_cluster['cluster_id'])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Normalized Entity Cluster Size')

    plt.title(f'({get_title_key()}) Better predicted by AnyBURL per Question Entity Cluster with Normalized Entity Cluster Size (n={n})')
    plt.xlabel('Cluster Id')
    plt.ylabel('Better predicted by AnyBURL')
    plt.ylim(-1,1)

    plt_show()   

In [ ]:
def show_better_predicted_per_answer_cluster(df, dataset_name, use_difference_psi, bp_threshold, n):
    """
    expects a df with 't_id', 'rank_filtered_anyburl' and 'rank_filtered_kge'
    """
    df = get_better_predicted_by(df, use_difference_psi, bp_threshold)
    entity_cluster = get_entity_clusters(
        os.path.join('experiments', f'{dataset_name}_{subsymbolic}_1', 'checkpoint_best.pt'), n
    )[['h_id', 't_id', 'cluster_id', 'freq', 'norm_freq']]
    
    entity_cluster_h = entity_cluster[['h_id', 'cluster_id', 'freq', 'norm_freq']].rename(columns={'cluster_id': 'cluster_id_h','freq':'freq_h', 'norm_freq':'norm_freq_h'})
    entity_cluster_t = entity_cluster[['t_id', 'cluster_id', 'freq', 'norm_freq']].rename(columns={'cluster_id': 'cluster_id_t','freq':'freq_t', 'norm_freq':'norm_freq_t'})
    
    df_w_cluster = df.join(entity_cluster_t.set_index('t_id')).join(entity_cluster_h.set_index('h_id'))
    
    df_w_cluster.loc[(df_w_cluster['predicted_head'] == True),'cluster_id'] = df_w_cluster.loc[(df_w_cluster['predicted_head'] == True),'cluster_id_h']
    df_w_cluster.loc[(df_w_cluster['predicted_head'] == False),'cluster_id'] = df_w_cluster.loc[(df_w_cluster['predicted_head'] == False),'cluster_id_t']
    
    df_w_cluster.loc[(df_w_cluster['predicted_head'] == True),'freq'] = df_w_cluster.loc[(df_w_cluster['predicted_head'] == True),'freq_h']
    df_w_cluster.loc[(df_w_cluster['predicted_head'] == False),'freq'] = df_w_cluster.loc[(df_w_cluster['predicted_head'] == False),'freq_t']
    
    df_w_cluster.loc[(df_w_cluster['predicted_head'] == True),'norm_freq'] = df_w_cluster.loc[(df_w_cluster['predicted_head'] == True),'norm_freq_h']
    df_w_cluster.loc[(df_w_cluster['predicted_head'] == False),'norm_freq'] = df_w_cluster.loc[(df_w_cluster['predicted_head'] == False),'norm_freq_t']
    
    
    df_by_cluster = df_w_cluster[['cluster_id', f'better_predicted_by_anyburl_t{bp_threshold}', 'freq', 'norm_freq']].groupby('cluster_id').mean()
    df_by_cluster.reset_index(inplace=True)
    df_by_cluster = df_by_cluster.sort_values(f'better_predicted_by_anyburl_t{bp_threshold}')
    
    cmap = plt.cm.viridis
    norm = matplotlib.colors.Normalize(vmin=df_by_cluster['norm_freq'].min(), vmax=df_by_cluster['norm_freq'].nlargest(100).values[0])

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.bar(range(len(df_by_cluster['cluster_id'])), df_by_cluster[f'better_predicted_by_anyburl_t{bp_threshold}'], color=cmap(norm(df_by_cluster['norm_freq'].values)))
    
    ax.set_xticks(range(len(df_by_cluster['cluster_id'])), df_by_cluster['cluster_id'])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    clb = fig.colorbar(sm)
    clb.set_label('Normalized Entity Cluster Size')

    plt.title(f'({get_title_key()}) Better predicted by AnyBURL per Answer Entity Cluster with Normalized Entity Cluster Size (n={n})')
    plt.xlabel('Cluster Id')
    plt.ylabel('Better predicted by AnyBURL')
    plt.ylim(-1,1)

    plt_show()   